In [21]:
import pandas as pd
import csv
import pymongo
import json
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [2]:
us_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
us_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
global_cases = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
global_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
global_recovered = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

In [3]:
us_confirmed_df = pd.read_csv(us_confirmed)
us_deaths_df = pd.read_csv(us_deaths)
global_cases_df = pd.read_csv(global_cases)
global_deaths_df = pd.read_csv(global_deaths)
global_recovered_df = pd.read_csv(global_recovered)

## US DATA

In [43]:
states_data = pd.DataFrame(
    us_deaths_df[
        ["UID", "iso2", "Admin2", "Province_State", "Population", "Lat", "Long_"]
    ]
).rename(columns={"iso2": "Country_Abbrv", "Admin2": "County", "Long_": "Long"})
states_data

,UID,Country_Abbrv,County,Province_State,Population,Lat,Long
0,16,AS,NaN,American Samoa,55641,-14.2710,-170.1320
1,316,GU,NaN,Guam,164229,13.4443,144.7937
2,580,MP,NaN,Northern Mariana Islands,55144,15.0979,145.6739
3,630,PR,NaN,Puerto Rico,2933408,18.2208,-66.5901
4,850,VI,NaN,Virgin Islands,107268,18.3358,-64.8963
...,...,...,...,...,...,...,...
3248,84090053,US,Unassigned,Washington,0,0.0000,0.0000
3249,84090054,US,Unassigned,West Virginia,0,0.0000,0.0000
3250,84090055,US,Unassigned,Wisconsin,0,0.0000,0.0000
3251,84090056,US,Unassigned,Wyoming,0,0.0000,0.0000


In [51]:
state_info = states_data.to_dict(orient ="records")
state_info

[{'UID': 16,
  'Country_Abbrv': 'AS',
  'County': nan,
  'Province_State': 'American Samoa',
  'Population': 55641,
  'Lat': -14.271,
  'Long': -170.132},
 {'UID': 316,
  'Country_Abbrv': 'GU',
  'County': nan,
  'Province_State': 'Guam',
  'Population': 164229,
  'Lat': 13.4443,
  'Long': 144.7937},
 {'UID': 580,
  'Country_Abbrv': 'MP',
  'County': nan,
  'Province_State': 'Northern Mariana Islands',
  'Population': 55144,
  'Lat': 15.0979,
  'Long': 145.6739},
 {'UID': 630,
  'Country_Abbrv': 'PR',
  'County': nan,
  'Province_State': 'Puerto Rico',
  'Population': 2933408,
  'Lat': 18.2208,
  'Long': -66.5901},
 {'UID': 850,
  'Country_Abbrv': 'VI',
  'County': nan,
  'Province_State': 'Virgin Islands',
  'Population': 107268,
  'Lat': 18.3358,
  'Long': -64.8963},
 {'UID': 84001001,
  'Country_Abbrv': 'US',
  'County': 'Autauga',
  'Province_State': 'Alabama',
  'Population': 55869,
  'Lat': 32.53952745,
  'Long': -86.64408227},
 {'UID': 84001003,
  'Country_Abbrv': 'US',
  'Count

In [49]:
us_cases_clean = pd.melt(
    us_confirmed_df.drop(
        columns=[
            "iso2",
            "iso3",
            "code3",
            "FIPS",
            "Admin2",
            "Province_State",
            "Country_Region",
            "Lat",
            "Long_",
            "Combined_Key",
        ]
    ),
    id_vars="UID",
    var_name="Date",
).rename(columns={"value": "Confirmed_Cases"})

us_cases_clean

,UID,Date,Confirmed_Cases
0,16,1/22/20,0
1,316,1/22/20,0
2,580,1/22/20,0
3,630,1/22/20,0
4,850,1/22/20,0
...,...,...,...
247223,84090053,4/6/20,648
247224,84090054,4/6/20,0
247225,84090055,4/6/20,0
247226,84090056,4/6/20,0


In [15]:
us_deaths_clean = pd.melt(
    us_deaths_df.drop(
        columns=[
            "iso2",
            "iso3",
            "code3",
            "FIPS",
            "Admin2",
            "Province_State",
            "Country_Region",
            "Lat",
            "Long_",
            "Combined_Key",
            "Population",
        ]
    ),
    id_vars="UID",
    var_name="Date",
).rename(columns={"value": "Deaths"})

us_deaths_clean

,UID,Date,Deaths
0,16,1/22/20,0
1,316,1/22/20,0
2,580,1/22/20,0
3,630,1/22/20,0
4,850,1/22/20,0
...,...,...,...
247223,84090053,4/6/20,0
247224,84090054,4/6/20,0
247225,84090055,4/6/20,0
247226,84090056,4/6/20,0


In [7]:
us_covid_data = pd.merge(
    us_cases_clean, us_deaths_clean, how="left", on=["UID", "Date"]
)
us_covid_data

,UID,Date,Confirmed_Cases,Deaths
0,16,1/22/20,0,0
1,316,1/22/20,0,0
2,580,1/22/20,0,0
3,630,1/22/20,0,0
4,850,1/22/20,0,0
...,...,...,...,...
247223,84090053,4/6/20,648,0
247224,84090054,4/6/20,0,0
247225,84090055,4/6/20,0,0
247226,84090056,4/6/20,0,0


In [52]:
us_covid_info = us_covid_data.to_dict(orient ="records")
us_covid_info

[{'UID': 16, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 316, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 580, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 630, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 850, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 84001001, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 84001003, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 84001005, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 84001007, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 84001009, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 84001011, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 84001013, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 84001015, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Deaths': 0},
 {'UID': 84001017, 'Date': '1/22/20', 'Confirmed_Cases': 0, 'Dea

## GLOBAL DATA

In [8]:
global_cases_clean = pd.melt(
    global_cases_df,
    id_vars=["Province/State", "Country/Region", "Lat", "Long"],
    var_name="Date",
).rename(columns={"value": "Confirmed_Cases"})

global_cases_clean

,Province/State,Country/Region,Lat,Long,Date,Confirmed_Cases
0,NaN,Afghanistan,33.00000,65.000000,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0
...,...,...,...,...,...,...
19983,Falkland Islands (Islas Malvinas),United Kingdom,-51.79630,-59.523600,4/6/20,2
19984,Saint Pierre and Miquelon,France,46.88520,-56.315900,4/6/20,1
19985,NaN,South Sudan,6.87700,31.307000,4/6/20,1
19986,NaN,Western Sahara,24.21550,-12.885800,4/6/20,4


In [9]:
global_deaths_clean = pd.melt(
    global_deaths_df,
    id_vars=["Province/State", "Country/Region", "Lat", "Long"],
    var_name="Date",
).rename(columns={"value": "Deaths"})

global_deaths_clean

,Province/State,Country/Region,Lat,Long,Date,Deaths
0,NaN,Afghanistan,33.00000,65.000000,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0
...,...,...,...,...,...,...
19983,Falkland Islands (Islas Malvinas),United Kingdom,-51.79630,-59.523600,4/6/20,0
19984,Saint Pierre and Miquelon,France,46.88520,-56.315900,4/6/20,0
19985,NaN,South Sudan,6.87700,31.307000,4/6/20,0
19986,NaN,Western Sahara,24.21550,-12.885800,4/6/20,0


In [10]:
global_recovered_clean = pd.melt(
    global_recovered_df,
    id_vars=["Province/State", "Country/Region", "Lat", "Long"],
    var_name="Date",
).rename(columns={"value": "Recovered"})

global_recovered_clean

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.00000,65.000000,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0
...,...,...,...,...,...,...
18919,Falkland Islands (Islas Malvinas),United Kingdom,-51.79630,-59.523600,4/6/20,0
18920,Saint Pierre and Miquelon,France,46.88520,-56.315900,4/6/20,0
18921,NaN,South Sudan,6.87700,31.307000,4/6/20,0
18922,NaN,Western Sahara,24.21550,-12.885800,4/6/20,0


In [11]:
global_covid_data = pd.merge(
    pd.merge(
        global_cases_clean,
        global_deaths_clean,
        how="left",
        on=["Province/State", "Country/Region", "Lat", "Long", "Date"],
    ),
    global_recovered_clean,
    how="left",
    on=["Province/State", "Country/Region", "Lat", "Long", "Date"],
)
global_covid_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed_Cases,Deaths,Recovered
0,NaN,Afghanistan,33.00000,65.000000,1/22/20,0,0,0.0
1,NaN,Albania,41.15330,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0,0,0.0
...,...,...,...,...,...,...,...,...
19983,Falkland Islands (Islas Malvinas),United Kingdom,-51.79630,-59.523600,4/6/20,2,0,0.0
19984,Saint Pierre and Miquelon,France,46.88520,-56.315900,4/6/20,1,0,0.0
19985,NaN,South Sudan,6.87700,31.307000,4/6/20,1,0,0.0
19986,NaN,Western Sahara,24.21550,-12.885800,4/6/20,4,0,0.0
